In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 线性SVM

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [ ]:
# 1.获取数据集
iris = load_iris()
X = iris.data
y = iris.target

# 2.1.数据预处理，构建二分类数据
X = X[y < 2, :2]
y = y[y < 2]

# 2.2.涉及求距离，注意对数据进行标准化处理
standardScaler = StandardScaler()
X_standard = standardScaler.fit_transform(X)

In [ ]:
def plot_decision_boundary(model, axis):
    """绘制决策边界"""
    x0, x1 = np.meshgrid(
        np.linspace(axis[0], axis[1], int((axis[1]-axis[0])*100)),
        np.linspace(axis[2], axis[3], int((axis[3]-axis[2])*100))
    )
    X_new = np.c_[x0.ravel(), x1.ravel()]

    y_predict = model.predict(X_new)
    zz = y_predict.reshape(x0.shape)

    from matplotlib.colors import ListedColormap
    custom_cmap = ListedColormap(['#EF9A9A', '#FFF59D', '#90CAF9'])

    plt.contourf(x0, x1, zz, cmap=custom_cmap)


def plot_svc_decision_boundary(model, axis):
    """绘制SVC决策边界+支撑向量"""
    plot_decision_boundary(model, axis)

    # 绘制一条决策边界的支撑向量
    w = model.coef_[0]
    b = model.intercept_[0]

    # w0 * x0 + w1 * x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    plot_x = np.linspace(axis[0], axis[1], 200)
    up_y = -w[0]/w[1] * plot_x - b/w[1] + 1/w[1]
    down_y = -w[0]/w[1] * plot_x - b/w[1] - 1/w[1]

    # 过滤 y，防止超出绘制范围
    up_index = (up_y >= axis[2]) & (up_y <= axis[3])
    down_index = (down_y >= axis[2]) & (down_y <= axis[3])

    plt.plot(plot_x[up_index], up_y[up_index], color='black')
    plt.plot(plot_x[down_index], down_y[down_index], color='black')


def plot_scatter(X, y):
    """绘制数据点"""
    plt.scatter(X[y == 0, 0], X[y == 0, 1])
    plt.scatter(X[y == 1, 0], X[y == 1, 1])


def linearsvcTest(X, y, C=1e9):
    """线性SVC"""
    svc = LinearSVC(C=C, random_state=17)  # C越大，容错能力越小，不允许数据点落在支撑向量之间
    svc.fit(X, y)

    plot_svc_decision_boundary(svc, axis=[-3, 3, -3, 3])
    plot_scatter(X, y)
    plt.show()

## Hard Margin SVM
- 会考虑每个数据点，要求数据是**线性可分的**，支持向量之间不允许有数据点
- 个别的干扰数据点，会影响整体分类，降低模型泛化能力

In [ ]:
linearsvcTest(X_standard, y)

## Soft Margin SVM
- **引入超参数C**，调节容错范围(**容错边界，位于支撑向量内部**)，允许数据点落在支撑向量与容错边界之间

In [ ]:
linearsvcTest(X_standard, y, C=100000)

In [ ]:
linearsvcTest(X_standard, y, C=1)

In [ ]:
linearsvcTest(X_standard, y, C=0.01)

# 非线性SVM

In [ ]:
from sklearn import datasets

In [ ]:
plot_scatter(*datasets.make_moons())

In [ ]:
X, y = datasets.make_moons(noise=0.15, random_state=666)
plot_scatter(X, y)

## 使用多项式特征的SVM

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC


def PolynomialSVC(degree, C=1.0):
    """多项式SVC"""
    return Pipeline([
        ("ploy", PolynomialFeatures(degree)),    # 1.转换为线性回归, degree: 多项式的阶数
        ("std_scaler", StandardScaler()),        # 2.数据集标准化
        ("linear_svc", LinearSVC(C=C))           # 3.线性SVM
    ])


def PolynomialSVCTest(degree, C=1.0):
    poly_svc = PolynomialSVC(degree, C)
    poly_svc.fit(X, y)
    plot_decision_boundary(poly_svc, axis=[-1.5, 2.5, -1.0, 1.5])
    plot_scatter(X, y)

In [ ]:
PolynomialSVCTest(degree=3, C=100)

In [ ]:
PolynomialSVCTest(degree=3)

In [ ]:
PolynomialSVCTest(degree=3, C=0.1)

## 使用多项式核函数的SVM
- **引入超参数degree**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


def PolynomialKernelSVC(degree, C=1.0):
    """多项式核SVC"""
    return Pipeline([
        ("std_scaler", StandardScaler()),
        ("kernel_svc", SVC(kernel="poly", degree=degree, C=C))
    ])


def PolynomialKernelSVCTest(degree, C=1.0):
    poly_kernel_svc = PolynomialKernelSVC(degree, C)
    poly_kernel_svc.fit(X, y)
    plot_decision_boundary(poly_kernel_svc, axis=[-1.5, 2.5, -1.0, 1.5])
    plot_scatter(X, y)

In [ ]:
PolynomialKernelSVCTest(degree=3, C=100)

In [ ]:
PolynomialKernelSVCTest(degree=3)

In [ ]:
PolynomialKernelSVCTest(degree=3, C=0.001)

## 使用高斯核函数的SVM
### 直观理解高斯核函数
- 二维坐标轴的含义:
  - 回归问题，x轴表示的是特征，y轴表示的是结果
  - 分类为题，x、y轴表示的都是特征

In [ ]:
def plot_rbf_svc_2d():
    # =====================================================
    # 步骤一 : 生成一维线性不可分特征数据
    # (1)数据点分布在 x 方向上，无法找到一个分界点，区分两类数据
    # (2)只有一个特征，是一维数据，分布在 x 方向，无 y 轴
    # =====================================================
    x = np.arange(-4, 5, 1)
    y = np.array((x >= -2) & (x <= 2), dtype=int)
    plt.scatter(x[y == 0], 0 * y[y == 0])
    plt.scatter(x[y == 1], 0 * y[y == 1])
    plt.show()
    
    # =======================================================
    # 步骤二 : 将一维数据映射到二维空间
    # (1)升维，将低维线性不可分的数据，变成高维线性可分
    # (2)高斯核会将m*n的数据映射成m*m，便于可视化，这里映射成二维
    # =======================================================
    def gaussion(x, l):
        gamma=1.0
        return np.exp(-gamma * (x - l) ** 2)  # x,l 是数字


    l1, l2 = -1, 1                        # 固定 2 个地标   <-- (l1, l2)
    X_new = np.zeros((len(x), 2))         # 生成 2 维特征   <-- (x1, x2)
    for i, data in enumerate(x):
        X_new[i, 0] = gaussion(data, l1)  # 与地标 l1 运算  <-- x1
        X_new[i, 1] = gaussion(data, l2)  # 与地标 l2 运算  <-- x2

    plt.scatter(X_new[y == 0, 0], X_new[y == 0, 1])
    plt.scatter(X_new[y == 1, 0], X_new[y == 1, 1])
    plt.plot([-0.1, 0.2], [0.2, -0.1], '--', color='red')
    plt.show()

In [ ]:
plot_rbf_svc_2d()

### 高斯核
- **引入超参数gamma**，调节模型复杂度。gamma越大，模型越复杂，越容易过拟合

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


def RBFKernelSVC(gamma=1.0):
    """RBF核SVC"""
    return Pipeline([
        ("std_scaler", StandardScaler()),
        ("svc", SVC(kernel="rbf", gamma=gamma))
    ])


def RBFKernelSVCTest(gamma=1.0):
    rbf_kernel_svc = RBFKernelSVC(gamma=gamma)
    rbf_kernel_svc.fit(X, y)
    plot_decision_boundary(rbf_kernel_svc, axis=[-1.5, 2.5, -1.0, 1.5])
    plot_scatter(X, y)

In [ ]:
RBFKernelSVCTest(gamma=0.01)

In [ ]:
RBFKernelSVCTest(gamma=0.1)

In [ ]:
RBFKernelSVCTest(gamma=0.5)

In [ ]:
RBFKernelSVCTest(gamma=1)

In [ ]:
RBFKernelSVCTest(gamma=10)

In [ ]:
RBFKernelSVCTest(gamma=100)

# SVM解决回归问题

In [ ]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR  # 线性
from sklearn.svm import SVR  # 线性核
from sklearn.metrics import r2_score
import sys
import numpy as np
import matplotlib.pyplot as plt


def linear_svr_test():
    # ============================================================
    # SVM模型 : 线性、多项式、多项式核、高斯核...
    # SVM回归 : 使数据点尽可能多的落在支撑向量之间，引入超参数 epsilon
    # ============================================================
    def StandardLinearSVR(epsilon=0.1, C=1.0):
        """线性SVR"""
        return Pipeline([
            ("std_scaler", StandardScaler()),
            ("linear_svr", LinearSVR(C=C, epsilon=epsilon))
        ])

    def PolynomialSVR(epsilon=0.1, degree=2, C=1.0):
        """多项式SVR"""
        return Pipeline([
            ("ploy", PolynomialFeatures(degree)),
            ("std_scaler", StandardScaler()),
            ("linear_svc", LinearSVR(C=C, epsilon=epsilon))
        ])

    def PolynomialKernelSVR(epsilon=0.1, degree=2, C=1.0):
        """多项式核SVR"""
        return Pipeline([
            ("std_scaler", StandardScaler()),
            ("poly_svr", SVR(kernel="poly", degree=degree, C=C, epsilon=epsilon))
        ])

    def RBFKernelSVR(epsilon=0.1, gamma=1.0):
        """RBF核SVR"""
        return Pipeline([
            ("std_scaler", StandardScaler()),
            ("rbf_svr", SVR(kernel="rbf", gamma=gamma, epsilon=epsilon))
        ])

    def run(X_train, X_test, y_train, y_test, svr_model, title=''):
        # 训练、预测
        svr_model.fit(X_train, y_train)
        y_predict = svr_model.predict(X_test)

        # 绘制预测结果
        plt.scatter(np.arange(len(y_test)), y_test, color='g')
        plt.scatter(np.arange(len(y_predict)), y_predict, color='r', marker='+')
        plt.title(label=title)
        plt.show()
        print("R square: ", r2_score(y_test, y_predict))

    # ===================
    # 步骤一 : 加载数据集
    # ===================
    boston = load_boston()
    X = boston.data
    y = boston.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

    # ===================
    # 步骤二: 测试 svc
    # ===================
    run(X_train, X_test, y_train, y_test, StandardLinearSVR(),   "线性SVR")
    run(X_train, X_test, y_train, y_test, PolynomialSVR(),       "多项式SVR")
    run(X_train, X_test, y_train, y_test, PolynomialKernelSVR(), "多项式核SVR")
    run(X_train, X_test, y_train, y_test, RBFKernelSVR(),        "RBF核SVR")

In [ ]:
linear_svr_test()